In [1]:
import numpy as np
import scipy as sp
import os 
import sys
import importlib

In [2]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_steven_free_utils as msfreeu
import my_data_process_utils as mdpu
import my_meal_detection_utils as mmdu
import my_cluster_utils as mclusteru
#importlib.reload(mdu)

In [3]:
annots = mfileu.read_file('data', 'free_data_steven_annots.pkl')
free_data = mfileu.read_file('data', 'free_data_steven_right.pkl')

In [7]:
def get_result(subj, sess, minMealDistance, minMealDuration, plot=False):
    results = []
    
    a = annots[subj][sess]        
    a = msfreeu.process_anntos(len(free_data[subj][sess]), a)
    
    proba = all_proba_bite[subj][sess]
    indices = proba[:, 0] + 40
    proba = proba[:, -1]
    proba_th = np.percentile(proba, 99.9)
    bite_indices = find_events(proba, indices, proba_th=proba_th)        

    proba = all_proba_free_window[subj][sess]
    indices = proba[:, 0] + 40
    proba = proba[:, -1]
    proba_th = np.percentile(proba, 99.9)
    meal_indices = find_events(proba, indices, proba_th=proba_th)        

    
    clusters = mclusteru.cluster_events_by_minute_steven(bite_indices, meal_indices)
    clusters = mclusteru.process_clusters(clusters, minMealDistance=minMealDistance, minMealDuration=minMealDuration)
    cond = (clusters[:, 2]>=1 ) | ((clusters[:, 1] - clusters[:,0])>=5*16*60)
    clusters = clusters[cond]    
    
    result = mmdu.get_meal_detection_results(a, clusters=clusters)
    if plot:
        plot_result(a, clusters, bite_indices=bite_indices, meal_indices=[], title="Subj, Sess: "+str(subj)+", "+str(sess))
    
    return result

In [8]:
def get_metric_results(minMealDistance, minMealDuration, plotFlag=False):
    gts, acovs, clcovs = [], [], []
    for subj in range(len(annots)):
        for sess in range(len(annots[subj])):
            
            gt, ac, clc = get_result(subj, sess, minMealDistance=minMealDistance, minMealDuration=minMealDuration, plot=plotFlag)
            assert len(gt) == len(ac)

            gt = mdpu.add_subj_sess_to_array(gt, subj, sess, at_begin=False)
            ac = mdpu.add_subj_sess_to_array(ac, subj, sess, at_begin=False)
            clc = mdpu.add_subj_sess_to_array(clc, subj, sess, at_begin=False)

            gts = gt if len(gts)==0 else np.concatenate((gts, gt))
            acovs = ac if len(acovs)==0 else np.concatenate((acovs, ac))
            clcovs = clc if len(clcovs)==0 else np.concatenate((clcovs, clc))
            
    res = mmdu.get_metric_results(gts, acovs, clcovs)
    return res

In [9]:
res = get_metric_results(minMealDistance=5*60*16, minMealDuration=60*16)
for key, val in res.items():
    print(key, " : ", val)

total  :  82
total_meal  :  49
total_snack  :  33
tp  :  69
tp_meal  :  45
tp_snack  :  24
recall  :  0.8414634146341463
recall_meal  :  0.9183673469387755
recall_snack  :  0.7272727272727273
fp  :  73
precision  :  0.4859154929577465
f1  :  0.6160714285714286
start_error  :  102.08003048780488
start_error_meal  :  103.59693877551021
start_error_snack  :  99.82765151515152
end_error  :  266.1234756097561
end_error_meal  :  340.5459183673469
end_error_snack  :  155.61742424242425
fragment_error  :  0.08536585365853659
fragment_error_meal  :  0.14285714285714285
fragment_error_snack  :  0.0


In [10]:
res ={}
for minMealDistance in range(0, 31, 5): #in minute
    for minMealDuration in range(0, 301, 5): #in second
        print(minMealDistance, minMealDuration, end=' | ')
        res[(minMealDistance, minMealDuration)] = get_metric_results(minMealDistance = minMealDistance*60*16, minMealDuration=minMealDuration*16)

mfileu.write_file('final_results', 'steven.pkl', res)

0 0 | 0 5 | 0 10 | 0 15 | 0 20 | 0 25 | 0 30 | 0 35 | 0 40 | 0 45 | 0 50 | 0 55 | 0 60 | 0 65 | 0 70 | 0 75 | 0 80 | 0 85 | 0 90 | 0 95 | 0 100 | 0 105 | 0 110 | 0 115 | 0 120 | 0 125 | 0 130 | 0 135 | 0 140 | 0 145 | 0 150 | 0 155 | 0 160 | 0 165 | 0 170 | 0 175 | 0 180 | 0 185 | 0 190 | 0 195 | 0 200 | 0 205 | 0 210 | 0 215 | 0 220 | 0 225 | 0 230 | 0 235 | 0 240 | 0 245 | 0 250 | 0 255 | 0 260 | 0 265 | 0 270 | 0 275 | 0 280 | 0 285 | 0 290 | 0 295 | 0 300 | 5 0 | 5 5 | 5 10 | 5 15 | 5 20 | 5 25 | 5 30 | 5 35 | 5 40 | 5 45 | 5 50 | 5 55 | 5 60 | 5 65 | 5 70 | 5 75 | 5 80 | 5 85 | 5 90 | 5 95 | 5 100 | 5 105 | 5 110 | 5 115 | 5 120 | 5 125 | 5 130 | 5 135 | 5 140 | 5 145 | 5 150 | 5 155 | 5 160 | 5 165 | 5 170 | 5 175 | 5 180 | 5 185 | 5 190 | 5 195 | 5 200 | 5 205 | 5 210 | 5 215 | 5 220 | 5 225 | 5 230 | 5 235 | 5 240 | 5 245 | 5 250 | 5 255 | 5 260 | 5 265 | 5 270 | 5 275 | 5 280 | 5 285 | 5 290 | 5 295 | 5 300 | 10 0 | 10 5 | 10 10 | 10 15 | 10 20 | 10 25 | 10 30 | 10 35 | 10 40 

KeyboardInterrupt: 

In [ ]:
#for key, val in res[(60, 300)].items():
#    print(key, " : ", val)